In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from sklearn.preprocessing import StandardScaler
from torchvision import transforms

# --- 0. 重新执行数据准备步骤 ---
# (确保这个 Notebook 能独立运行)

# 请确保路径正确
DATA_DIR = './' # 你的数据文件夹路径
IMAGE_DIR = DATA_DIR # 你的图片文件夹路径

df = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
df_wide = pd.pivot_table(df, 
                         index=['image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm'], 
                         columns='target_name', 
                         values='target',
                         aggfunc='mean').reset_index()
df_wide = df_wide.rename_axis(None, axis=1)
df_wide['Sampling_Date'] = pd.to_datetime(df_wide['Sampling_Date'])


# --- 1. 定义图像预处理/增强 ---
# 对于验证集，我们只做基础的尺寸调整、Tensor转换和归一化
# 对于训练集，可以加入随机翻转、颜色抖动等数据增强
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(), # 随机水平翻转
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNet 均值和标准差
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


# --- 2. 自定义 PyTorch 数据集类 ---
class BiomassDataset(Dataset):
    def __init__(self, dataframe, image_dir, tabular_features, target_cols, transform=None):
        """
        Args:
            dataframe (pd.DataFrame): 包含所有信息的宽格式 DataFrame.
            image_dir (str): 图像文件所在的目录.
            tabular_features (pd.DataFrame): 经过预处理的表格特征.
            target_cols (list): 目标列的列名列表.
            transform (callable, optional): 应用于图像的 torchvision 变换.
        """
        self.df = dataframe
        self.image_dir = image_dir
        self.transform = transform
        
        # 提取表格特征和目标值
        self.tabular_data = tabular_features.values.astype(np.float32)
        # !! 关键：同样在这里进行 log 变换 !!
        self.targets = np.log1p(self.df[target_cols].values.astype(np.float32))

    def __len__(self):
        # 返回数据集的总样本数
        return len(self.df)

    def __getitem__(self, idx):
        # 根据索引 idx 获取单个样本
        
        # 1. 加载图像
        img_name = self.df.iloc[idx]['image_path']
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        # 2. 应用图像变换
        if self.transform:
            image = self.transform(image)
        
        # 3. 获取对应的表格数据和目标
        tabular_row = torch.tensor(self.tabular_data[idx], dtype=torch.float)
        target_row = torch.tensor(self.targets[idx], dtype=torch.float)
        
        return image, tabular_row, target_row

print("--- PyTorch 环境和 Dataset 类定义完成 ---")
print(f"PyTorch 版本: {torch.__version__}")
print("BiomassDataset 类已准备就绪。")


--- PyTorch 环境和 Dataset 类定义完成 ---
PyTorch 版本: 2.5.1
BiomassDataset 类已准备就绪。


In [2]:
import torch
import torch.nn as nn
import torchvision.models as models

class BiModalModel(nn.Module):
    def __init__(self, num_tabular_features, num_targets=5, pretrained=True):
        """
        Args:
            num_tabular_features (int): 输入的表格特征数量.
            num_targets (int): 需要预测的目标数量 (本项目中是 5).
            pretrained (bool): 是否使用预训练的 CNN 权重.
        """
        super(BiModalModel, self).__init__()
        
        # --- 1. 图像分支 (Image Branch) ---
        # 加载一个预训练的 ResNet18 模型
        self.cnn = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1 if pretrained else None)
        
        # 获取 ResNet18 最后一层全连接层 (fc) 的输入特征数
        num_cnn_features = self.cnn.fc.in_features
        
        # 将原始的 fc 层替换为一个 Identity 层，相当于只做特征提取，不做分类
        self.cnn.fc = nn.Identity()
        
        # --- 2. 表格分支 (Tabular Branch) ---
        self.tabular_mlp = nn.Sequential(
            nn.Linear(num_tabular_features, 128),
            nn.BatchNorm1d(128), # BatchNorm 有助于稳定训练
            nn.ReLU(),
            nn.Dropout(0.3), # Dropout 防止过拟合
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        # --- 3. 融合与最终预测 (Fusion Head) ---
        # 将 CNN 特征和 MLP 特征拼接后的总维度
        total_features = num_cnn_features + 64
        
        self.fusion_head = nn.Sequential(
            nn.Linear(total_features, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_targets) # 最终输出 5 个预测值
        )

    def forward(self, image, tabular):
        # 定义数据如何流过网络
        
        # 1. 图像数据通过 CNN
        image_features = self.cnn(image)
        
        # 2. 表格数据通过 MLP
        tabular_features = self.tabular_mlp(tabular)
        
        # 3. 拼接 (Concatenate) 特征
        combined_features = torch.cat((image_features, tabular_features), dim=1)
        
        # 4. 通过融合层得到最终输出
        output = self.fusion_head(combined_features)
        
        return output

print("--- 多模态模型 BiModalModel 定义完成 ---")
# 我们可以创建一个模型实例来测试一下结构是否正确
# (这里我们先假设表格特征有 21 个，和第二阶段一样)
test_model = BiModalModel(num_tabular_features=21)
print("模型结构:")
print(test_model)



--- 多模态模型 BiModalModel 定义完成 ---
模型结构:
BiModalModel(
  (cnn): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-0

In [4]:
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm # 引入 tqdm 来显示进度条

# --- 1. 最终数据准备 ---
# a) 复制一份数据
df_train_val = df_wide.copy()

# b) 特征工程 (与第二阶段完全一致)
df_train_val['Month'] = df_train_val['Sampling_Date'].dt.month
df_train_val['Month_sin'] = np.sin(2 * np.pi * df_train_val['Month'] / 12)
df_train_val['Month_cos'] = np.cos(2 * np.pi * df_train_val['Month'] / 12)
df_train_val = pd.get_dummies(df_train_val, columns=['State', 'Species'], drop_first=True)

target_cols = ['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'GDM_g', 'Dry_Total_g']
feature_cols = [col for col in df_train_val.columns if col not in target_cols + ['image_path', 'Sampling_Date', 'Month']]

# c) 将数据划分为训练集 (80%) 和验证集 (20%)
train_df, val_df = train_test_split(df_train_val, test_size=0.2, random_state=42)

# d) 对数值表格特征进行标准化 (Standardization)
# !! 关键：只能用训练集的数据来 fit scaler，然后用它来 transform 训练集和验证集 !!
scaler = StandardScaler()
numerical_cols = ['Pre_GSHH_NDVI', 'Height_Ave_cm', 'Month_sin', 'Month_cos']
train_tabular_features = train_df[feature_cols].copy()
val_tabular_features = val_df[feature_cols].copy()

train_tabular_features[numerical_cols] = scaler.fit_transform(train_df[numerical_cols])
val_tabular_features[numerical_cols] = scaler.transform(val_df[numerical_cols])


# --- 2. 创建 Datasets 和 DataLoaders ---
BATCH_SIZE = 16 # 每个批次加载 16 张图片

train_dataset = BiomassDataset(train_df.reset_index(drop=True), IMAGE_DIR, train_tabular_features, target_cols, transform=data_transforms['train'])
val_dataset = BiomassDataset(val_df.reset_index(drop=True), IMAGE_DIR, val_tabular_features, target_cols, transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)


print(f"数据准备完成。训练集样本: {len(train_dataset)}, 验证集样本: {len(val_dataset)}")


# --- 3. 设置训练组件 ---
# a) 确定设备 (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"将使用设备: {device}")

# b) 实例化模型并移动到设备
model = BiModalModel(num_tabular_features=len(feature_cols)).to(device)

# --- !! 新增代码：冻结 CNN 骨干网络 !! ---
# 1. 先冻结所有 CNN 的参数
for param in model.cnn.parameters():
    param.requires_grad = False

# 2. 只有我们自己添加的 MLP 和 Fusion Head 部分的参数才需要训练
#    优化器只会更新 requires_grad = True 的参数
print("--- CNN 骨干网络已冻结，只训练顶层分类器 ---")
# ---------------------------------------------

# c) 定义损失函数和优化器
criterion = nn.MSELoss() 
# 优化器现在只会收到需要更新的参数
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3, weight_decay=1e-4)


# --- 4. 训练与验证循环 ---
NUM_EPOCHS = 20 # 训练 20 个轮次
best_rmse = float('inf') # 记录最好的 RMSE

for epoch in range(NUM_EPOCHS):
    # --- 训练阶段 ---
    model.train() # 设置为训练模式
    running_loss = 0.0
    
    # 使用 tqdm 显示进度条
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [训练]")
    for images, tabular, targets in pbar:
        images, tabular, targets = images.to(device), tabular.to(device), targets.to(device)
        
        optimizer.zero_grad() # 清空梯度
        outputs = model(images, tabular) # 前向传播
        loss = criterion(outputs, targets) # 计算损失
        loss.backward() # 反向传播
        optimizer.step() # 更新权重
        
        running_loss += loss.item() * images.size(0)
    
    epoch_train_loss = running_loss / len(train_dataset)

    # --- 验证阶段 ---
    model.eval() # 设置为评估模式
    all_preds = []
    all_targets = []
    
    with torch.no_grad(): # 在此模式下，不计算梯度，节省计算资源
        pbar_val = tqdm(val_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [验证]")
        for images, tabular, targets in pbar_val:
            images, tabular, targets = images.to(device), tabular.to(device), targets.to(device)
            outputs = model(images, tabular)
            
            # !! 关键：将预测值和真实值都从 log 尺度还原 !!
            preds_orig = np.expm1(outputs.cpu().numpy())
            targets_orig = np.expm1(targets.cpu().numpy())
            
            all_preds.append(preds_orig)
            all_targets.append(targets_orig)
            
    # 计算整个验证集的 RMSE
    val_rmse = np.sqrt(mean_squared_error(np.concatenate(all_targets), np.concatenate(all_preds)))
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} -> 训练损失: {epoch_train_loss:.4f} | 验证 RMSE: {val_rmse:.4f}")
    
    # 保存表现最好的模型
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"  -> 新的最佳模型已保存，RMSE: {best_rmse:.4f}")

print("\n--- 训练完成 ---")
print(f"最好的验证集 RMSE 是: {best_rmse:.4f}")



数据准备完成。训练集样本: 285, 验证集样本: 72
将使用设备: cpu
--- CNN 骨干网络已冻结，只训练顶层分类器 ---


Epoch 1/20 [训练]:   0%|          | 0/18 [00:00<?, ?it/s]

Epoch 1/20 [验证]:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/20 -> 训练损失: 5.5965 | 验证 RMSE: 31.0248
  -> 新的最佳模型已保存，RMSE: 31.0248


Epoch 2/20 [训练]:   0%|          | 0/18 [00:00<?, ?it/s]

...

Epoch 19/20 [验证]:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 19/20 -> 训练损失: 0.4343 | 验证 RMSE: 15.9289
  -> 新的最佳模型已保存，RMSE: 15.9289


Epoch 20/20 [训练]:   0%|          | 0/18 [00:00<?, ?it/s]

Epoch 20/20 [验证]:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 20/20 -> 训练损失: 0.4245 | 验证 RMSE: 17.1840

--- 训练完成 ---
最好的验证集 RMSE 是: 15.9289


结果出来了！这是一个**非常、非常有价值**的结果！

让我们把三次实验的结果放在一起，做一个最终的对比。

### 第三阶段：最终成果总结

| 实验策略 | 模型 | 最终表现 (最好的验证 RMSE) |
| :--- | :--- | :--- |
| **阶段二 (基准)** | **LightGBM (仅表格)** | **11.7058** |
| 阶段三 (第一次尝试) | 多模态 (完全训练) | 23.5757 |
| **阶段三 (第二次尝试)** | **多模态 (冻结骨干)** | **15.9289** |

---

### 结果解读与最终结论

#### 1. 好的方面：我们的策略是成功的！
*   **“冻结”策略效果显著**：通过冻结 ResNet18 的骨干网络，我们的模型表现从 `23.57` 大幅提升到了 `15.92`。这是一个巨大的进步！
*   **证明了之前的诊断**：这完全证实了我们之前的判断——**过拟合**是导致模型表现不佳的罪魁祸首。通过大幅减少需要训练的参数，我们有效地缓解了过拟合，让模型学到了更有用的知识。

#### 2. “坏”的方面：我们仍然没有战胜基准
*   尽管取得了巨大进步，但我们深度学习模型的最佳成绩 `15.9289`，仍然比只用表格数据的 LightGBM 模型的 `11.7058` 要**差很多**。

---

### 核心结论：这个项目教会了我们什么？

> **在当前的数据量和模型复杂度下，图像信息带来的价值，不足以弥补其引入的额外复杂度和噪声。表格特征 (`NDVI`, `Height` 等) 在这个任务中占据了主导地位。**

这是一个在真实世界的数据科学项目中非常常见的、也是非常有价值的结论。**并非总是越复杂的模型效果越好**，尤其是在数据量有限的情况下。一个简单、鲁棒的梯度提升树模型（如 LightGBM）往往能战胜一个没有被充分优化的复杂深度学习模型。

### 接下来我们还能做什么？(进阶探索)

我们已经完成了这个项目的核心流程，并得出了清晰的结论。如果你还想继续深入探索，这里有几个可以尝试的方向：

#### **方向一：微调 (Fine-tuning) - 最经典的下一步**
*   **原理**：我们已经把新加的“头”训练好了。现在，我们可以“解冻”整个网络（或者只解冻 ResNet 的最后几层），然后用一个**非常非常小**的学习率（比如 `lr=1e-5` 或 `1e-6`）来训练整个模型。这叫做“微调”，目的是在不破坏预训练权重的基础上，让整个网络更适应我们的特定数据。
*   **这是最有可能进一步提升 DL 模型性能的方法。**

#### **方向二：更换更强的CNN骨干**
*   我们用的是 `ResNet18`，可以换成更现代、更强大的 `EfficientNet-B0` 或 `EfficientNet-B1` 试试，看看它们能否提取出更有价值的图像特征。

#### **方向三：回到并优化基准模型**
*   既然 LightGBM 效果最好，我们可以把精力放在优化它上面。比如进行**超参数调优**（调整树的数量、深度、学习率等），它的分数可能还能进一步降低。

#### **方向四：接受当前结果**
*   在很多实际业务场景中，如果一个简单模型已经能很好地解决问题，我们就会选择它，因为它训练更快、更容易部署和解释。**接受“LightGBM 是此任务的最佳模型”这个结论，本身就是一次成功的项目实践。**
